In [5]:
!ls /n/holylabs/LABS/kempner_fellows/Users/binxuwang/DL_Projects/mini_edm/exps/BigBlnrX3_RAVEN10_abstract_20240305-2338

checkpoints	 loss_curve.png  samples_inferred_rule_consistency.npz
dataset_idx.pkl  samples	 std.log


In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from os.path import join

expdir = r'/n/holylabs/LABS/kempner_fellows/Users/binxuwang/DL_Projects/mini_edm/exps/BigBlnrX3_RAVEN10_abstract_20240305-2338'
# dataset_idx = np.load(join(expdir, 'dataset_idx.pkl'))
dataset_idx = pkl.load(open(join(expdir, 'dataset_idx.pkl'), 'rb'))

In [5]:
!pwd

/n/home12/binxuwang/Github/DiffusionReasoning/notebooks


In [6]:

from functools import partial
import os 
from os.path import join
from tqdm import trange, tqdm
import glob
import time
import copy
import random
from PIL import Image, ImageOps, ImageFilter
import re
import pandas as pd
import einops
import numpy as np 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as pltb
import sys
sys.path.append('/n/home12/binxuwang/Github/DiffusionReasoning')
from edm_utils import parse_train_logfile
from dataset_utils import onehot2attr_tsr
from stats_plot_utils import estimate_CI, shaded_error
from rule_utils import get_rule_list, get_obj_list, get_rule_img, check_consistent

def batch_load_samples_infer_rules(samples_dir, epoch_list, encoding="onehot", fmt="tensor_%s.pt"):
    rules_all = []
    for epoch in tqdm(epoch_list): 
        if not os.path.exists(join(samples_dir, fmt % epoch)):
            print(epoch, "not exist")
            break
        samples = torch.load(join(samples_dir, fmt % epoch)) # (batch, 27, 9, 9)
        if encoding == "onehot":
            attr_tsr_list = onehot2attr_tsr(samples, threshold=0.5)
        elif encoding == "digit":
            attr_tsr_list = torch.round(samples).int() # (batch, 3, 9, 9)
        else:
            raise ValueError("encoding should be onehot or digit")
        rules_list = []
        for i, attr_tsr in enumerate(attr_tsr_list): 
            rule_img = get_rule_img(attr_tsr) # (3, 9, 9) -> (3,)
            rules_list.append(rule_img)
        rules_all.append(rules_list)
    rules_all = np.asarray(rules_all) # (201, 25, 3)
    epoch_list = epoch_list[:len(rules_all)]
    
    consistent_mat = []
    for epoch_i in trange(len(rules_all)): 
        consistent_all = [check_consistent(rules) 
                          for rules in rules_all[epoch_i]]
        consistent_mat.append(consistent_all)
    consistent_mat = np.asarray(consistent_mat)
    return rules_all, consistent_mat, epoch_list


In [7]:
sample_dir= "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results/008-RAVEN10_abstract-DiT_S_3/samples/"

rules_all, consistent_mat, epoch_list = batch_load_samples_infer_rules(sample_dir, epoch_list=[682000,], encoding="digit", fmt="%07d.pt")

100%|██████████| 1/1 [00:00<00:00, 161.20it/s]


In [11]:
sample_dir= "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results/008-RAVEN10_abstract-DiT_S_3/samples/"

rules_all, consistent_mat, epoch_list = batch_load_samples_infer_rules(sample_dir, epoch_list=[683000,], encoding="digit", fmt="%07d.pt")

  0%|          | 0/1 [00:00<?, ?it/s]


683000 not exist


0it [00:00, ?it/s]


In [15]:
len(rules_all)

0

In [9]:
rules_all.shape

(1, 1024, 3)

In [17]:
sorted(os.listdir("/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results"))

['000-RAVEN10_abstract-DiT_S_1',
 '001-RAVEN10_abstract-DiT_B_1',
 '002-RAVEN10_abstract-DiT_S_1',
 '003-RAVEN10_abstract_onehot-DiT_S_1',
 '004-RAVEN10_abstract-DiT_S_1',
 '005-RAVEN10_abstract_onehot-DiT_S_1',
 '006-RAVEN10_abstract-DiT_S_1',
 '007-RAVEN10_abstract_onehot-DiT_S_1',
 '008-RAVEN10_abstract-DiT_S_3',
 '009-RAVEN10_abstract-DiT_S_1',
 '010-RAVEN10_abstract-DiT_L_1',
 '011-RAVEN10_abstract-DiT_B_1',
 '012-RAVEN10_abstract-DiT_B_3',
 '013-RAVEN10_abstract-DiT_S_3',
 '014-RAVEN10_abstract_onehot-DiT_S_3',
 '015-RAVEN10_abstract_onehot-DiT_B_3',
 '016-RAVEN10_abstract-DiT_S_3',
 '017-RAVEN10_abstract_onehot-DiT_S_1',
 '018-RAVEN10_abstract-DiT_S_1',
 '019-RAVEN10_abstract_onehot-DiT_S_1',
 '020-RAVEN10_abstract-DiT_S_1',
 '021-RAVEN10_abstract-uncond-DiT_S_3_20240308-1349',
 '022-RAVEN10_abstract_onehot-uncond-DiT_B_3_20240308-1349',
 '023-RAVEN10_abstract_onehot-uncond-DiT_S_3_20240308-1349',
 '024-RAVEN10_abstract-uncond-DiT_B_3_20240308-1349',
 '025-RAVEN10_abstract_oneho

In [2]:
np.asarray([[0,0,0]])

array([[0, 0, 0]])

In [8]:
dataset_idx.keys()

dict_keys(['row_ids', 'y'])

In [9]:
dataset_idx['row_ids']

tensor([[1088, 6569, 2527],
        [4894, 4650, 5613],
        [9876, 6563, 4420],
        ...,
        [3171,  578, 6258],
        [6818, 5413, 6417],
        [5928, 4555,   35]])

In [10]:
dataset_idx['y']

tensor([ 0,  0,  0,  ..., 34, 34, 34], dtype=torch.int32)

In [ ]:
import torch
from torch.utils.data import Dataset
import einops
import einops
from tqdm import trange, tqdm
from torch.utils.data import Dataset, DataLoader
def _sample_panels(train_row_img, cmb_per_class=3333):
    """
    train_row_img: [35, 10000, 3, 3, 9] 
    """
    X = []
    y = []
    row_ids = []
    n_classes = train_row_img.shape[0]
    n_samples = train_row_img.shape[1]
    for iclass in trange(n_classes):
        tuple_loader = DataLoader(range(n_samples), batch_size=3, shuffle=True, drop_last=True)
        X_class = []
        row_ids_cls = []
        while True:
            try:
                batch = next(iter(tuple_loader))
                rows = train_row_img[iclass][batch]
                mtg = torch.cat(tuple(rows), dim=1)
                X_class.append(mtg)
                row_ids_cls.append(batch)
                if len(X_class) == cmb_per_class:
                    break
            except StopIteration:
                tuple_loader = DataLoader(range(n_samples), batch_size=3, shuffle=True, drop_last=True)

        y_class = torch.tensor([iclass]*len(X_class), dtype=torch.int)
        X_class = torch.stack(X_class)
        row_ids_cls = torch.stack(row_ids_cls)
        y.append(y_class)
        X.append(X_class)
        row_ids.append(row_ids_cls)
    X = torch.cat(X, dim=0)
    y = torch.cat(y, dim=0)
    row_ids = torch.cat(row_ids, dim=0)
    return X, y, row_ids


def reconstruct_imgs(X, row_ids, ys):
    """ """
    img_all = []
    for iclass, batch_id in zip(ys, row_ids):
        rows = X[iclass][batch_id]
        mtg = torch.cat(tuple(rows), dim=1)
        img_all.append(mtg)
    img_all = torch.stack(img_all)
    return img_all


class dataset_PGM_abstract(Dataset): 
    def __init__(self, cmb_per_class=3333, train_attrs=None, device="cpu", onehot=False): 
        """attr_list: [num_samples, 3, 9, 3]"""
        if train_attrs is None:
            train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs.pt') # [35, 10000, 3, 9, 3]
        n_classes = train_attrs.shape[0] # 35
        n_samples = train_attrs.shape[1] # 10k
        self.labels = torch.arange(0, n_classes).unsqueeze(1).expand(n_classes, n_samples)
        train_attrs = train_attrs.to(int)
        self.train_row_img = einops.rearrange(train_attrs, 'c s pnl (H W) attr -> c s attr H (pnl W)', H=3, W=3, attr=3, pnl=3)
        self.X, self.y, self.row_ids = _sample_panels(self.train_row_img, cmb_per_class)
        self.X = self.X.to(device) # [35 * cmb_per_class, 3, 9, 9]
        if onehot is True:
            O1 = torch.cat([torch.eye(7, 7, dtype=int), torch.zeros(1, 7, dtype=int)], dim=0)
            O2 = torch.cat([torch.eye(10, 10, dtype=int), torch.zeros(1, 10, dtype=int)], dim=0)
            O3 = torch.cat([torch.eye(10, 10, dtype=int), torch.zeros(1, 10, dtype=int)], dim=0)
            X_onehot = torch.cat([O1[self.X[:, 0], :], O2[self.X[:, 1], :], O3[self.X[:, 2], :], ], dim=-1)
            print(X_onehot.shape)
            self.X = einops.rearrange(X_onehot, 'b h w C -> b C h w')
            print(self.X.shape)
            self.Xmean = torch.tensor([0.5, ]).view(1, 1, 1, 1)
            self.Xstd = torch.tensor([0.5, ]).view(1, 1, 1, 1)
            self.X = (self.X.float() - self.Xmean) / self.Xstd
        else:
            self.Xmean = torch.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to(device)
            self.Xstd = torch.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to(device)
            self.X = (self.X - self.Xmean) / self.Xstd
        
    def __len__(self): 
        return len(self.y)
    
    def __getitem__(self, idx): 
        """attr: [3, 9, 3]"""
        return self.X[idx], self.y[idx]
    
    def dict(self):
        return {'row_ids': self.row_ids, 'y': self.y}


In [16]:
import torch
import einops
train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs.pt')
train_row_img = einops.rearrange(train_attrs, 'c s pnl (H W) attr -> c s attr H (pnl W)', H=3, W=3, attr=3, pnl=3)
train_row_img.shape

torch.Size([35, 10000, 3, 3, 9])

In [17]:
expdir = r'/n/holylabs/LABS/kempner_fellows/Users/binxuwang/DL_Projects/mini_edm/exps/BigBlnrX3_RAVEN10_abstract_20240305-2338'
# dataset_idx = np.load(join(expdir, 'dataset_idx.pkl'))
dataset_idx = pkl.load(open(join(expdir, 'dataset_idx.pkl'), 'rb'))

In [24]:
def reconstruct_imgs(X, row_ids, ys):
    """ """
    img_all = []
    for iclass, batch_id in zip(ys, row_ids):
        rows = X[iclass][batch_id]
        mtg = torch.cat(tuple(rows), dim=1)
        img_all.append(mtg)
    img_all = torch.stack(img_all)
    return img_all

X_all = reconstruct_imgs(train_row_img, dataset_idx['row_ids'], dataset_idx['y']).to(torch.int)

In [25]:
X_all.shape

torch.Size([116655, 3, 9, 9])

In [40]:
def rule_summary_table(rules_all, consistent_mat, epoch_list):
    rule_summary_df = []
    assert rules_all.shape[0] == consistent_mat.shape[0] == len(epoch_list)
    for rule_ep, consistent_ep, epoch in zip(rules_all, consistent_mat, epoch_list): 
        rule_valid = (rule_ep != -1).mean()
        rule_valid_cnt = (rule_ep != -1).sum()
        rule_consistent_1 = (consistent_ep == 1).mean()
        rule_consistent_1_cnt = (consistent_ep == 1).sum()
        rule_consistent_2 = (consistent_ep == 2).mean()
        rule_consistent_2_cnt = (consistent_ep == 2).sum()
        rule_consistent_3 = (consistent_ep == 3).mean()
        rule_consistent_3_cnt = (consistent_ep == 3).sum()
        rule_summary_df.append({"epoch": epoch,
                                "valid": rule_valid.mean(),
                                "valid_cnt": rule_valid_cnt,
                                "cst_1": rule_consistent_1.mean(),
                                "cst_1_cnt": rule_consistent_1_cnt,
                                "cst_2": rule_consistent_2.mean(),
                                "cst_2_cnt": rule_consistent_2_cnt,
                                "cst_3": rule_consistent_3.mean(),
                                "cst_3_cnt": rule_consistent_3_cnt,
                                })
    rule_summary_df = pd.DataFrame(rule_summary_df)
    print(rule_summary_df.tail())
    return rule_summary_df

In [38]:
import sys
sys.path.append('/n/home12/binxuwang/Github/DiffusionReasoning')
from edm_utils import parse_train_logfile
from dataset_utils import onehot2attr_tsr
from stats_plot_utils import estimate_CI, shaded_error
from rule_utils import get_rule_list, get_obj_list, get_rule_img, check_consistent
from tqdm import trange, tqdm

def batch_infer_rule(X, threshold=0.5, encoding="onehot"):
    """ 
    X: [batch, 3, 9, 9]
    """
    rules_list = []
    for attr_tsr in trange(X): 
        rule_img = get_rule_img(attr_tsr) # (3, 9, 9) -> (3,)
        rules_list.append(rule_img)
    rules_all = np.asarray(rules_list)
    # consistent_mat = []
    # for epoch_i in trange(len(rules_all)): 
    consistent_all = [check_consistent(rules) 
                        for rules in rules_all]
        # consistent_mat.append(consistent_all)
    consistent_mat = np.asarray(consistent_all)
    return rules_all, consistent_mat

rules_all_train, consistent_mat_train = batch_infer_rule(X_all, threshold=0.5, encoding="digit")

In [45]:
rule_summary_df = rule_summary_table(rules_all_train[None,:], consistent_mat_train[None,:], [None])

  epoch     valid  valid_cnt     cst_1  cst_1_cnt     cst_2  cst_2_cnt  \
0  None  0.644073     225403  0.571411      66658  0.077356       9024   

      cst_3  cst_3_cnt  
0  0.059346       6923  


In [46]:
np.savez(join(expdir, 'train_set_rule_summary.npz'), rules_all_train=rules_all_train, consistent_mat_train=consistent_mat_train)
rule_summary_df.to_csv(join(expdir, 'train_set_rule_summary.csv'))